In [10]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
import os

In [2]:
mp_holistic = mp.solutions.holistic # inicia el modelo donde podemos seguir los puntos del cuerpo humano, incluyendo pose, mano y caras.
mp_drawing = mp.solutions.drawing_utils # inicializa el dibujado o trazados de los puntos del modelo.
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh

In [3]:
def mediapipe_detection(image, model):
    """
    OpenCV carga las imágenes en el formato BGR por defecto. Sin embargo, 
    muchos modelos de procesamiento de imágenes, 
    incluido MediaPipe, esperan que las imágenes estén en formato RGB. 
    Por lo tanto, esta funcion convierte la imagen del formato BGR a RGB y devuelve la posición de
    los puntos  clave del cuerpo, la mano y la cara.
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_styled_landmarks(image,results):
    """
        La función permite conectar los puntes de referenica a la imagen y luego dibuja
        las conexiones, permitiendo ver (mallas).
    """
    # Dibuja los landmarks del cuerpo
    mp_drawing.draw_landmarks(image, #Toma como referencia la imagen capturada
                                  results.pose_landmarks, #Toma como referencia los puntos predecidos por el modelo Holistic.
                                  mp_pose.POSE_CONNECTIONS, #Conecta cada punto predecidos.
                                  mp_drawing.DrawingSpec( #Funcion que personaliza los puntos predecidos.
                                                         color=(80,22,10), #Color del punto
                                                         thickness=2, #Grosor del punto
                                                         circle_radius=4), #Radio del circulo.
                                  mp_drawing.DrawingSpec( #Funcion que personaliza las conexiones predecidas.
                                                         color=(80,44,121),
                                                         thickness=2,
                                                         circle_radius=2))
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1))
    mp_drawing.draw_landmarks(image, results.face_landmarks,mp_face_mesh.FACEMESH_CONTOURS,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_hands.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_hands.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))

In [5]:
def extract_keypoints(results):
    face = np.array([[res.x,res.y,res.z]for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    rh = np.array([[res.x,res.y,res.z]for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    lh = np.array([[res.x,res.y,res.z]for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    pose = np.array([[res.x,res.y,res.z,res.visibility]for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return np.concatenate([pose,face,lh,rh])

In [6]:
no_videos = 1
no_frames = 30

In [7]:
category = {
    'VOCALES': ['A','E','I','O','U'],
    'PRONOMBRES':['TU','YO','EL','ELLA'],
    'DIAS':['LUNES','JUEVES','VIERNES'],
}
actions = np.array([value for values in category.values() for value in values])

In [8]:
model = load_model('models/train8.h5')

In [19]:
sequence = []
for no_frame in range(no_frames):
    # res = np.load(os.path.join('Directorio','Seña','Num video',"{}.npy".format(no_frame)))
    # res = np.load(os.path.join('MP_Data_Celular','ELLA','8',"{}.npy".format(no_frame)))
    res = np.load(os.path.join('MP_Data_Webcam','YO','8',"{}.npy".format(no_frame)))
    sequence.append(res)

In [ ]:
cap = cv2.VideoCapture(0)
superBrake = False
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while True:
        ret,frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image,results)
        cv2.imshow('Recolectando Datos',image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cv2.waitKey(1000)
    sequence = []
    for frame_num in range(no_frames):
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image,results)
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)

        if frame_num + 1 == no_frames:
            cv2.putText(image,'PRESIONE CUALQUIERA TECLA PARA SALIR',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2,cv2.LINE_AA)
            cv2.putText(image,'RECOLECTANDO VIDEO...',(15,35),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
            cv2.putText(image,'FRAME CAPTURADO #{}'.format(frame_num+1),(15,60),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
            # cv2.imshow('Recolectando Datos',image)
            cv2.waitKey(0)
        else: 
            cv2.putText(image,'RECOLECTANDO VIDEO...',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
            cv2.putText(image,'FRAME CAPTURADO #{}'.format(frame_num+1),(15,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
            # cv2.imshow('Recolectando Datos',image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
        cv2.imshow('Recolectando Datos',image)
        
    cap.release()
    cv2.destroyAllWindows()

In [12]:
sequence

[array([ 0.51419765,  0.35570154, -1.00724185, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.50815284,  0.35716444, -0.8308441 , ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.50656122,  0.35817605, -0.84406537, ...,  0.        ,
         0.        ,  0.        ]),
 array([ 0.5067277 ,  0.35823017, -0.77957362, ...,  0.20981273,
         0.62430024, -0.02228922]),
 array([ 0.50694156,  0.35844091, -0.8165493 , ...,  0.20690204,
         0.6121065 , -0.02008724]),
 array([ 0.50713325,  0.35851541, -0.76690954, ...,  0.20582663,
         0.60663301, -0.02086015]),
 array([ 0.50738895,  0.35712829, -0.70295626, ...,  0.20279023,
         0.60599869, -0.01931718]),
 array([ 0.50847524,  0.35464886, -0.72803175, ...,  0.19947278,
         0.61587816, -0.02047384]),
 array([ 0.51129168,  0.35424006, -0.75768977, ...,  0.16385698,
         0.64740688, -0.00529589]),
 array([ 5.13047636e-01,  3.54007512e-01, -7.70014107e-01, ...,
         1.96228594e-01,  6

In [13]:
np.array(sequence).shape

(30, 1662)

In [20]:
if len(sequence) == no_frames:
    res = model.predict(np.expand_dims(sequence, axis=0))[0]
    print(np.argmax(res))
    print(actions[np.argmax(res)])
else:
    print("La secuencia no tiene la longitud requerida.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
6
YO
